## Метрики в задаче регрессии 📏
Как гласит одна замечательная фраза: "Если ты не знаешь как что-то измерить, то ты не можешь это улучшить". Действительно, сложно спорить. Дело в том, что цель любой ML-модели - осуществлять верные предсказания, чтобы хорошо решать определенную бизнес задачу. Я рекомендую избегать результаты экспериментов, где просто говорят, что модель "хорошо" или "адекватно" работает. Что значит "хорошо" или "плохо"? Это сколько? 

Как видишь, такие термины лишены деталей, которые усложняют процесс интерпретации и улучшения модели. Вместо этого было бы идеально описывать результаты своих экспериментов фразами:
- "В среднем модель ошибается на 10.000 руб. при прогнозировании стоимости автомобиля"
- "В среднем модель ошибается на 15% от истинной цены услуги" 
- ...

Именно для этого и используются метрики.

- Метрика — это некоторая математическая формула, позволяющая оценить насколько хорошо модель выполняет свою задачу. Например, точность предсказаний или качество классификации.

Метрик в ML очень много, но в рамках этого ноутбука мы сфокусируемся на основных метркиах для решения задачи регрессии. Многие метрики уже имплементированы в sklearn в разделе [metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html)

In [19]:
import numpy as np
import pandas as pd
import warnings

SEED = 23
warnings.filterwarnings('ignore')

## Данные 📊
Я предлагаю использовать несложный датасет по оценке стоимостей автомобилей. Это неплохой вариант для нашей темы, так как целевая переменная будет содержать информацию о дешевых и дорогих автомобмилях, что отлично подходит для сравнения таких метрик как $MAE$ и $RMSE$. Датасет содержит информацию о ценах на автомобили, произведенные в США. Более подробно с датасетом можно ознакомиться [здесь](https://www.kaggle.com/datasets/hellbuoy/car-price-prediction?select=CarPrice_Assignment.csv).


In [27]:
data = pd.read_csv('../data/regression/usa_car_prices.csv')
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


- В датасете 26 фичей. Логично предположить, что не все из них важны и для построения лучше модели необходимо сделать отбор признаков - _feature selection_. В рамках нашей темы это будет избыточно, поэтому я просто сфокусируюсь на нескольких числовых признаках которые отберу по-своему предпочтению:
    - _Кол-во лошадиных сил | horsepower_
    - _Длина авто | carlength_
    - _Объем двигателя | enginesize_
    - _Вес авто_ | curbweight_
    - _Цена автомобиля | price_

In [28]:
# отбираем признаки
features = [
    'horsepower',
    'carlength',
    'enginesize',
    'curbweight',
    'price'
]

df = data[features]
df.head()

,horsepower,carlength,enginesize,curbweight,price
0,111,168.8,130,2548,13495.0
1,111,168.8,130,2548,16500.0
2,154,171.2,152,2823,16500.0
3,102,176.6,109,2337,13950.0
4,115,176.6,136,2824,17450.0


Давай обучим простую модель регресии и сравним прогнозы модели с известными значениями

In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# как всегда, вначале разобьем данные на train и test
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, shuffle=True)

# осуществляем масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# обучим модель
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# спрогнозируем на test
y_pred = model.predict(X_test_scaled)

# cформируем таблицу с истинными и предсказанными значениями
metrics_results = pd.DataFrame({'true': y_test, 'pred': y_pred.flatten()})
metrics_results['error'] = metrics_results['true'] - metrics_results['pred']
metrics_results.head()

,true,pred,error
74,45400.0,33459.814874,11940.185126
51,6095.0,5441.606425,653.393575
46,11048.0,12783.570040,-1735.570040
14,24565.0,18848.847159,5716.152841
18,5151.0,205.239611,4945.760389


Отлично, теперь у нас есть ошибка на каждом объекте выборки. Однако, мы ведь не можем судить о качестве модели, используя лишь 1 объект? Необходимо учитывать все объекты, какие есть варианты?
- _"Суммирование отклонения_": не подходит, так как произойдет компенсация ошибки. Например, если отклонения -2 и 2, то сумма равна 0. Получается что ошибки нет, однако это не так
- _"Квадрат отклонения_": подходит, компенсации ошибки не не произойдет
- _"Абсолютное отклонение_": подходит, компенсации ошибки ткакже не происходит

In [57]:
# давай посмотрим какие ошибки мы получим, используя методы выше
sum_error = metrics_results['error'].sum().item()
sum_squared_error= (metrics_results['error'] ** 2).sum().item()
sum_abs_error = metrics_results['error'].abs().sum().item()

print(f"Сумма отклонений: {sum_error:.2f}")
print(f"Сумма квадратов отклонений: {sum_squared_error:.2f}")
print(f"Сумма абсолютных отклонений: {sum_abs_error:.2f}")

Сумма отклонений: 67038.65
Сумма квадратов отклонений: 1078337927.81
Сумма абсолютных отклонений: 178092.24


Полученные ошибки - накопительные, а нам бы желательно понимать как, например, модель ошибается в среднем, поэтому давай просто усредним полученные значения. 

In [58]:
n = metrics_results.shape[0]
mean_sum_error = sum_error / n
mean_squared_error = sum_squared_error / n
mean_abs_error = sum_abs_error / n

print(f"Средняя сумма отклонений: {mean_sum_error:.2f}")
print(f"Средняя сумма квадратов отклонений: {mean_squared_error:.2f}")
print(f"Средняя сумма абсолютных отклонений: {mean_abs_error:.2f}")

Средняя сумма отклонений: 1081.27
Средняя сумма квадратов отклонений: 17392547.22
Средняя сумма абсолютных отклонений: 2872.46


Теперь полученные метрики отражают <u>усредненную</u> ошибку модели для всех объектов выборки, то что нам нужно! Как мы обсудили выше - некорректо использовать "среднюю сумму отклонения" из-за компенсации. Оставшивеся 2 метрики действительно используются в машинном обучении и называются соответственно _среднеквадратичная_ и _среднеабсолютная_ ошибки о которых мы поговорим далее

## Среднеквадратичная ошибка | Mean Squared Error | MSE
$$ MSE = \frac{1}{n}\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

$MSE$ — одна из самых популярных метрик для регрессионных моделей. Она измеряет средний квадрат разницы между реальными значениями $y_i$ и предсказаниями $\hat{y}_i$.

Интуитивно, $MSE$ показывает, как сильно в среднем ошибается модель. Давай взглянем на вид функциональной зависимсоти ошибки. Ну так как это квадратичная ошибка, то график должен быть похож на параболу

In [70]:
import plotly.express as px

# ошибки
error = metrics_results['error']
squared_error = metrics_results['error'] ** 2

# сортировка чтобы красиво отрисовалось
error_df = pd.DataFrame({
    "error": error,
    "squared_error": squared_error
}).sort_values("error")

fig = px.line(
    error_df,
    x="error",
    y="squared_error",
    title="Mean Squared Error",
    labels={"error": "Отклонение", "squared_error": "Ошибка"}
)

fig.add_scatter(
    x=error_df["error"], y=error_df["squared_error"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Квадратичная ошибка | Squared Error",
    showlegend=True
)

Хмм, ну и где парабола? Да, график похож на нее, но почему правая чсть длинее левой? В чем может быть причина?

<!-- Ответ
Дело в том, что распределение цены авто не является нормальным/симметричным - есть небольшое кол-во наблюдений с большой ценой. Следовательно, это приводит к большим положительным отклонениям которые делают распределение ошибки несимметричным. Скорее всего распределение таргета имеет длинный правый хвост, на котором модель занижает прогноз. Если бы в выборке было достаточно дорогих авто, модель научилась бы лучше предсказывать их стоимость, и хвост распределения ошибок был бы более симметричным.
-->

In [71]:
# твои мысли 🧠

Чтобы убедиться, что ошибка действительно симметричная давай зафиксируем диапазон отклонения, например будем рассматривать только от $[-5.5k, 5.5k]$

In [77]:
error_threshold = 5500
squared_error_threshold = 30*10**6

error_df_cut = error_df[error_df['error'] <= error_threshold]
error_df_cut = error_df_cut[error_df_cut['squared_error'] <= squared_error_threshold]

fig = px.line(
    error_df_cut,
    x="error",
    y="squared_error",
    title="Mean Squared Error",
    labels={"error": "Отклонение", "squared_error": "Ошибка"}
)

fig.add_scatter(
    x=error_df_cut["error"], y=error_df_cut["squared_error"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Квадратичная ошибка | Squared Error",
    showlegend=True
)

Вот теперь хорошо видно, что ошибка симметричаня. Заметь, что величина штрафа - нелинейная (квадртатичная). Выходит, чем сильнее ошибается модель, тем сильнее штраф. Отсюда мы встречаемся с первым минусом у $MSE$:

- $MSE$ очень чувствительна к выбросам!

Могу доказать на простом примере, смотри ...

In [129]:
# давай добавим 2 новых столбца - абсолютное отклонение и квадаратичное
metrics_results['error_abs'] = metrics_results['error'].abs()
metrics_results['error_squared'] = metrics_results['error'] ** 2

# найдем наблюдения с минимальным и максимальным абсолютными отклонениями
min_error_abs = metrics_results['error_abs'].min()
max_error_abs = metrics_results['error_abs'].max()

min_max_error_df = metrics_results[
    metrics_results['error_abs'].isin([min_error_abs, max_error_abs])
]

print(f"Минимальное абсолютное отклонение: {min_error_abs:.2f}")
print(f"Максимальное абсолютное отклонение: {max_error_abs:.2f}")

Минимальное абсолютное отклонение: 19.75
Максимальное абсолютное отклонение: 14495.02


Данные наблюдения идеально подходят для демонстрации чувствительности $MSE$ к выбросам. Так как модель очень сильно ошибается на втором объекте, то его можно смело посчитать за выброс. Наша исходная модель совершает вот таки квадартичные ошибки на данных объектах:

In [117]:
errors_old = min_max_error_df['error_squared'].values.tolist()
print('Ошибки:', errors_old)

Ошибки: [389.94176637864393, 210105511.76706648]


Тепреь предположим, что мы улучшили нашу модель - добавили больше признаков и качество симметрично улучшилось - стало точнее на 10$ (взял цифру с потолка <смеющийся смайлик>)

In [133]:
# красиво оформим DataFrame - как буто результат эксперимента модели A
columns = [
    'true',
    'pred',
    'error_squared',
    'error_abs'
]

model_a_results = min_max_error_df[columns].copy()
model_a_results.insert(0, 'model_version', 'A') # добавляем версию модели
model_a_results

,model_version,true,pred,error_squared,error_abs
157,A,7198.0,7217.746943,3.899418e+02,19.746943
128,A,37028.0,22532.983209,2.101055e+08,14495.016791


In [136]:
# результат эксперимента модели B
model_b_results = pd.DataFrame({
    'model_version': ['B', 'B'],
    'true': [7198.0, 37028.0],
    'pred': [7207.7, 22542.9], # предсказание теперь точнее на 10$ для обоих объектов
})

# считаем квадратичную и абсолютные ошибки
model_b_results['error_squared'] = (model_b_results['true'] - model_b_results['pred']) ** 2
model_b_results['error_abs'] = (model_b_results['true'] - model_b_results['pred']).abs()
model_b_results

,model_version,true,pred,error_squared,error_abs
0,B,7198.0,7207.7,9.409000e+01,9.7
1,B,37028.0,22542.9,2.098181e+08,14485.1


In [137]:
# теперь посчитаем на сколько уменьшилась квадратичная и абсолютная ошибки
error_reduction_squared = model_a_results['error_squared'].values - model_b_results['error_squared'].values
error_reduction_abs = model_a_results['error_abs'].values - model_b_results['error_abs'].values
print('Изменение квадратичной ошибки: ', error_reduction_squared.tolist())
print('Изменение абсолютной ошибки: ', error_reduction_abs.tolist())

Изменение квадратичной ошибки:  [295.8517663786475, 287389.75706651807]
Изменение абсолютной ошибки:  [10.04694321606894, 9.91679085148644]


Видно, что сильнее всего (почти на 300к) изменилась ошибка на выбросе (второе наблюдение), а не на "нормальном" объекте. Следовательно, модели будет всегда выгоднее точнее предсказывать выброс, ценой большей ошибки на "нормальных" данных. Таким образом, модель будет всегда подстраиваться под выбросы, чтобы достичь наименьшего $MSE$. Как думаешь хорошо это или плохо?

<!-- Ответ
Это зависи от бизнес задачи. Необходимо задать вопрос бизнесу - "Важно ли для нас хорошо предсказывать цену на выбросах или важна стабильная работа модели на основном сегменте цен?". Поэтому далее возможны 2 сценария
- Вариант А: Настраиваться на выбросы
    - Лучше будет предсказывать редкие наблюдения, однако хуже будет прогноз на основных наблюдениях - скорее всего будет завышать прогноз
- Вариант B: Игнорировать выбросы (их удаления или использование устойчивых функций потерь)
    - Модель будет ошибаться на выбросах - не сможет хорошо их предсказать - скорее всего будет занижать прогноз, однако на основных наблюдениях модель будет точна
-->

In [ ]:
# твои мысли 🧠

А что с абсолютной ошибкой? Как видишь, для этой метрики модели не важно на каком объекте улучшать прогноз, так как итоговые ошибки равны. Поэтому $MAE$ устойчив к выбросам! Давай перейдем к следующим 2-м минусам метрики $MSE$, связанные с еденицами измерения и интерпретируемостью.

#### _Проблема квадратных значений_

Так как значение ошибки возводится в квадрат, то при интерпретации ошибки мы будем оперировать квадратными еденицами - квадратные доллары/рубли/штуки, ... Представь, ты обучил модель, а затем говоришь заказчику, что твоя модель в среднем ошибается на 5000 квадратных долларов. Не совсем понятно как это понимать, не так ли? 

Избавить от квадрата очень просто - извелкаем корень из итоговой ошибки (спасибо курсу школьной математики <смайл смех>) и все, теперь квадратные доллары  это снова привычные и понятные нам доллары

#### _Проблема: ошибка в контексте - большая/маленькая?_

Также при расчете $MSE$ и последующего извлечения корня мы получаем ошибку значение которой не дает информации о критичности - большая или маленькая? Например, получил ты $RMSE = 5000$. Это много или мало? Один из вариантов это сбегать к заказчику и спросить. А если он в отпуске или не отвечает? Как тогда поступить? К счастью, можно учесть масштаб целевой переменной и получить новую метрику - "коэффициент детерминации R2" который легко интерпретируется.

#### _Минусы RMSE_

In [ ]:
# TODO

## Корень из среднеквадратичной ошибки | Root Mean Squared Error | RMSE
Мы уже затронули данную метрику выше - это просто квадратный корень из среднеквадратичной ошибки _MSE_

$$ RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2} $$

В отличие от $MSE$, величина $RMSE$ выражается в тех же единицах, что и целевая переменная. Это делает её более интерпретируемой: например, если мы предсказываем стоимость автомобилей в долларах, то $RMSE$ показывает среднюю ошибку прогноза в  настоящих долларах, вместо квадратных. Однако важно помнить, что само значение $RMSE$ не всегда говорит, много это или мало. Чтобы правильно оценить качество модели, необходимо сопоставлять $RMSE$ с разбросом целевой переменной - использовать метрику $R2$

$RMSE$ отличается от $MSE$ только тем, что в него добавлен квадратный корень. Метрика унаследовала все недостатки $MSE$, но при этом имеет одно важное преимущество — величина ошибки выражается в тех же единицах, что и сама целевая переменная $y$

## Коэффициент Детерменации $R2$
Это одна из наиболее популярных метрик качества регрессионных моделей. Как мы уже выясниили ранее, _RMSE_ не дает оценку критичности ошибки - большая или маленькая, поэтому если нормировать _"MSE"_ на дисперсию целевой перменной $y$, то получим абсолютно новую мерику - коэффициент детерменации, глядя на который можно легко понять на сколько хороша наша модель.

R2 показывает, как хорошо модель объясняет дисперсию целевой переменной $y$ 

$$ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}$$

- Числитель: используется не _MSE_, а _SSE - Sum Squared Error_, но интуация +/- как с _RMSE_
- Знаменатель: используется дисперсия таргета $y$, еще называют _SST - Sum of Squeres Total_

Выходит, что в идеале модель должна предсказывать либо идеально значения таргета, либо чтобы _SSE_ равнялся дисперсии $y$. Значение $R^2$ всегда лежит в интервале от $(-\infty, 1]$, так как это нормированная метрика. А вот так необходимо интерпретировать полученные результаты:
- $R^2 = 1$ - модель идеально предсказывает данные, без ошибок (редко бывает на практике)
- $R^2 = 0$ - модель ничем не лучше прогноза "средним" - "глупая модель" (бывает на практике, необходимо избегать)
- $R^2 < 0$ - модель хуже прогноза "средним" (бывает на практике, необходимо избегать)

На практике $R2$ для "разумной" модели обычно лежит на отрезке от $[0, 1]$. Чем ближе к 1 тем лучше! Если $R^2 < 0$, то с моделью 100% что то не так - недообучение/переобучение, выбросы, возможно _train_ и _test_ имеют разные распределения, ... В таких случаях необходим детальный анализ данных и ошибок модели. Хочу отметить, что формулу выше можно переписать в более интерсном виде:


$$ R^2 = 1 - \frac{SSE}{SST} = 1 - \frac{n\cdot RMSE^2}{SST}, RMSE = \sqrt{\frac{SSE}{n}}$$

$$R^2 = 1 - \left( \frac{RMSE_\text{model}}{RMSE_{\text{baseline}}} \right)^2, RMSE_{\text{baseline}} = \sqrt{\frac{\mathrm{SST}}{n}}$$

Фомулы выше доказывают, что $R2$ можно интерпретировать как показатель того, насколько модель предсказывает лучше, чем простая стратегия прогнозирования средним - часто используемая в качестве "baseline" решения.


In [151]:
# закодим метрику R2 и оценим качество нашей модели
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot) if ss_tot != 0 else 0

r2_score_test = r2_score(y_true=metrics_results['true'], y_pred=metrics_results['pred'])
print('Коэффициент детерминации R2 | Test:', r2_score_test)

Коэффициент детерминации R2 | Test: 0.8101000960701704


Получили довольно хороший результат, наша модель на 80% объясняет дисперсию целевой переменной. Лишь 20% отсутствует - можно подумать как усложнить модель. 

#### Минусы R2

_Зависимость от дисперсии целевой переменной_
- Если дисперсия таргета $y$ маленькая, то даже модель с хорошим $RMSE$ даст низкий $R2$, и наоборот, при большой дисперсии таргета даже не очень точная модель может показать высокий $R2$

_Чувствительность к выбросам_
- Даже несколько выбросов могут резко уронить $R2$

In [144]:
# давай снова посмотрим как будет выглядеть график ошибку для RMSE
error_df_cut['root_squared_error'] = error_df_cut['squared_error'] ** 0.5

fig = px.line(
    error_df_cut,
    x="error",
    y="root_squared_error",
    title="Mean Root Squared Error",
    labels={"error": "Отклонение", "root_squared_error": "Квадратный корень ошибки"}
)

fig.add_scatter(
    x=error_df_cut["error"], y=error_df_cut["root_squared_error"], mode="markers", name="points"
)

fig.update_layout(
    height=700, width=800,
    title_text="Корень из среднеквадратичной ошибки | Root Mean Squared Error",
    showlegend=True
)

In [34]:
# давай найдем самое максимальное и минималное абсолютные отклонения
min_error = metrics_results['error_abs'].min()
max_error = metrics_results['error_abs'].max()
print(f"Минимальное абсолютное отклонение: {min_error:.2f}")
print(f"Максимальное абсолютное отклонение: {max_error:.2f}")

metrics_results[metrics_results['error_abs'].isin([min_error, max_error])]

Минимальное абсолютное отклонение: 19.75
Максимальное абсолютное отклонение: 14495.02


,true,pred,error_abs
157,7198.0,7217.746943,19.746943
128,37028.0,22532.983209,14495.016791


Как видишь, есть наблюдения где модель практически не ошибается - очень маленькое отклонение, а есть объекты на которых ошибка просто коллосальная. Например, для нашищ данных ошибка на объекте с индексом 128 в 763 раза больше чем для объекта с индеком 157. Это очень много! Но почему так происходит? Дело в том, что цены на авто распределены ненормально - большинство наблюдений (авто) имеют небольшую стоимость, но есть небольшое количество наблюдений с большой стоимостью - дорогие авто. Так как дорогих авто в обучающей выборке значительно меньше бюджетных, то у модели появляется некоторый _bias_ - она начинает занижать цену автомобилей.  

Можно рассматривать дорогие автомобили как выбросы, которые непропорционально влияют на критерий качества и обучение модели. В таких условиях стандартная линейная регрессия часто ведёт себя плохо на хвостах распределения. К счастью, эту проблему можно решить при помощи специальных методов (логарифмирование, использование устойчивых функций потерь, ...)

In [14]:
# взглянем на истинное и предсказанное распределения
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

# Distribution plot
# считаем оценку плотности для фактических и предсказанных значений
kde_test = gaussian_kde(y_test)
kde_pred = gaussian_kde(y_pred)

# определим диапазон значений по которым бдуем строить плотности
min_value = min(y_test.min(), y_pred.min())
max_value = max(y_test.max(), y_pred.max())

# также задаим сколько точек будем строить на графике
n_points = 200

# итоговый диапазон значений
x_vals = np.linspace(min_value, max_value, n_points)

# строим график плотностей
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_vals, y=kde_test(x_vals),
    fill='tozeroy', name='Известные', mode='lines',
    line=dict(color='blue'), opacity=0.6
))

fig.add_trace(go.Scatter(
    x=x_vals, y=kde_pred(x_vals),
    fill='tozeroy', name='Прогноз', mode='lines',
    line=dict(color='red'), opacity=0.6
))

fig.update_layout(
    title='Сравнение плотностей распределений: Известные vs Прогноз | Test Data',
    xaxis_title='Sales', yaxis_title='Плотность',
    width=1000, height=600
)

fig.show()

* Сравнение распределений показывает зоны перепрогноза и недопрогноза. Теперь оценим качество модели с помощью метрик и разберём их плюсы и минусы.

## Среднеквадратичная ошибка | Mean Squared Error | MSE

## Корень из среднеквадратичной ошибки | Root Mean Squared Error | RMSE

## Среднеабсолютная ошибка | Mean Absolute Error | MAE

## Среднеабсолютная ошибка в процентах | Mean Absolute Percentage Error | MAPE

## MSLE

## Huber Loss

## Квантильная ошибка | Quantile Error/Loss